In [2]:
from Autolabeler import workout
import pandas as pd
import os
import warnings
import numpy as np
from pycaret.classification import *

# Load Test Data

In [14]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True)
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
data = workout(filelist=file_names[0:1], signals= signals)
df= pd.concat(data)

In [15]:
df.shape

(8184, 31)

In [16]:
X = df.drop(['peaks', 'first_sample','last_sample', 'first_sample_closest_peak', 
             'last_sample_closest_peak','exercise','exercising_periods'], axis=1)
X.columns

Index(['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'magnX', 'magnY',
       'magnZ', 'linAccX', 'linAccY', 'linAccZ', 'accX_mod', 'accY_mod',
       'accZ_mod', 'gyroX_mod', 'gyroY_mod', 'gyroZ_mod', 'magnX_mod',
       'magnY_mod', 'magnZ_mod', 'linAccX_mod', 'linAccY_mod', 'linAccZ_mod'],
      dtype='object')

# Extract SQUAT TIME RANGE

In [35]:
np.unique(df.exercising_periods, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25]),
 array([5341,  115,  126,  118,  132,  144,  125,  115,  129,  120,  108,
         116,  111,  111,  107,  105,  106,  102,  109,  101,  112,  105,
         108,  104,  106,  108], dtype=int64))

In [42]:
def extract_squat_time_range(df_workout):
    # Identify start and end times
    squats_detection = pd.concat([df_workout[ df_workout.first_sample == True ], df_workout[ df_workout.last_sample == True ]]).sort_index()
    
    start_times = squats_detection[squats_detection['first_sample']].index
    end_times = squats_detection[squats_detection['last_sample']].index
    
    # Assuming each start is followed by an end, pair them
    paired_times = list(zip(start_times, end_times))
    
    # Create new DataFrame
    return pd.DataFrame(paired_times, columns=['start', 'end'])

In [43]:
extract_squat_time_range(df)

,start,end
0,1970-01-01 00:00:00.474,1970-01-01 00:00:01.558
1,1970-01-01 00:00:03.012,1970-01-01 00:00:04.203
2,1970-01-01 00:00:05.666,1970-01-01 00:00:06.781
3,1970-01-01 00:00:08.396,1970-01-01 00:00:09.641
4,1970-01-01 00:00:11.172,1970-01-01 00:00:12.532
5,1970-01-01 00:00:14.170,1970-01-01 00:00:15.347
6,1970-01-01 00:00:16.757,1970-01-01 00:00:17.840
7,1970-01-01 00:00:19.277,1970-01-01 00:00:20.494
8,1970-01-01 00:00:22.092,1970-01-01 00:00:23.224
9,1970-01-01 00:00:24.916,1970-01-01 00:00:25.932


# Auxiliary Functions

In [18]:
def prepare_data(chunk_df):
    signals_name = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
    stats_dict = {}
    series = chunk_df[signals_name]
    for column in series.columns:
        mean = series[column].mean()
        std = series[column].std()
        median = series[column].median()
    
        stats_dict[f"{column}_mean"] = mean
        stats_dict[f"{column}_std"] = std
        stats_dict[f"{column}_median"] = median
        
    return pd.DataFrame.from_dict(stats_dict, orient="index").T

In [19]:
def predict_pycaret(pipeline_ml, ventana_df):
    output = prepare_data(ventana_df)
    prediction_df = predict_model(pipeline_ml, output, raw_score=True)
    y_pred_value = prediction_df.prediction_label.iloc[0]
    output['start'] = ventana.index[0]
    output['end'] = ventana.index[-1]
    output['prediction'] = y_pred_value
    return output[['start', 'end', 'prediction']]

# Load Model

In [20]:
# Load the model
pipeline = load_model(model_name="squats_traditional")

Transformation Pipeline and Model Successfully Loaded


In [21]:
ventana = prepare_data(X.iloc[0:170])
prediction = predict_model(pipeline, ventana, raw_score=True)
prediction.prediction_label.iloc[0]

'NO_EXERCISE'

# Check Data

In [22]:
tamano_ventana = 170 
nuevo_df = pd.DataFrame()
nuevo_df_predicciones = pd.DataFrame()

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    print(f"Ventana: {i} - {i + tamano_ventana}")
    ventana = X.iloc[i:i + tamano_ventana]
    
    X1 = predict_pycaret(pipeline, ventana)
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

Ventana: 0 - 170
Ventana: 170 - 340
Ventana: 340 - 510
Ventana: 510 - 680
Ventana: 680 - 850
Ventana: 850 - 1020
Ventana: 1020 - 1190
Ventana: 1190 - 1360
Ventana: 1360 - 1530
Ventana: 1530 - 1700
Ventana: 1700 - 1870
Ventana: 1870 - 2040
Ventana: 2040 - 2210
Ventana: 2210 - 2380
Ventana: 2380 - 2550
Ventana: 2550 - 2720
Ventana: 2720 - 2890
Ventana: 2890 - 3060
Ventana: 3060 - 3230
Ventana: 3230 - 3400
Ventana: 3400 - 3570
Ventana: 3570 - 3740
Ventana: 3740 - 3910
Ventana: 3910 - 4080
Ventana: 4080 - 4250
Ventana: 4250 - 4420
Ventana: 4420 - 4590
Ventana: 4590 - 4760
Ventana: 4760 - 4930
Ventana: 4930 - 5100
Ventana: 5100 - 5270
Ventana: 5270 - 5440
Ventana: 5440 - 5610
Ventana: 5610 - 5780
Ventana: 5780 - 5950
Ventana: 5950 - 6120
Ventana: 6120 - 6290
Ventana: 6290 - 6460
Ventana: 6460 - 6630
Ventana: 6630 - 6800
Ventana: 6800 - 6970
Ventana: 6970 - 7140
Ventana: 7140 - 7310
Ventana: 7310 - 7480
Ventana: 7480 - 7650
Ventana: 7650 - 7820
Ventana: 7820 - 7990
Ventana: 7990 - 8160


In [23]:
nuevo_df_predicciones.shape

(48, 3)

In [24]:
nuevo_df_predicciones.head()

,start,end,prediction
0,1970-01-01 00:00:00.000,1970-01-01 00:00:01.607,NO_EXERCISE
1,1970-01-01 00:00:01.616,1970-01-01 00:00:03.223,NO_EXERCISE
2,1970-01-01 00:00:03.232,1970-01-01 00:00:04.838,NO_EXERCISE
3,1970-01-01 00:00:04.847,1970-01-01 00:00:06.454,NO_EXERCISE
4,1970-01-01 00:00:06.467,1970-01-01 00:00:08.074,NO_EXERCISE


In [26]:
np.unique(nuevo_df_predicciones.prediction,return_counts=True)

(array(['NO_EXERCISE', 'SQUAT'], dtype=object), array([45,  3], dtype=int64))

In [27]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediction == 'SQUAT'][['start', 'end', 'prediction']]
squats_detection

,start,end,prediction
5,1970-01-01 00:00:08.083,1970-01-01 00:00:09.690,SQUAT
7,1970-01-01 00:00:11.315,1970-01-01 00:00:12.922,SQUAT
12,1970-01-01 00:00:19.402,1970-01-01 00:00:21.009,SQUAT


In [28]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediction == ('NO_EXERCISE'
                                                                              ''
                                                                              '')][['start', 'end', 'prediction']]
squats_detection

,start,end,prediction
0,1970-01-01 00:00:00.000,1970-01-01 00:00:01.607,NO_EXERCISE
1,1970-01-01 00:00:01.616,1970-01-01 00:00:03.223,NO_EXERCISE
2,1970-01-01 00:00:03.232,1970-01-01 00:00:04.838,NO_EXERCISE
3,1970-01-01 00:00:04.847,1970-01-01 00:00:06.454,NO_EXERCISE
4,1970-01-01 00:00:06.467,1970-01-01 00:00:08.074,NO_EXERCISE
6,1970-01-01 00:00:09.699,1970-01-01 00:00:11.306,NO_EXERCISE
8,1970-01-01 00:00:12.935,1970-01-01 00:00:14.542,NO_EXERCISE
9,1970-01-01 00:00:14.551,1970-01-01 00:00:16.157,NO_EXERCISE
10,1970-01-01 00:00:16.166,1970-01-01 00:00:17.773,NO_EXERCISE
11,1970-01-01 00:00:17.782,1970-01-01 00:00:19.389,NO_EXERCISE
